In [1]:
import torch
from torch.utils.data import Dataset

class MyTrainDataset(Dataset):
    def __init__(self, size):
        self.size = size
        self.data = [(torch.rand(20), torch.rand(1)) for _ in range(size)]

    def __len__(self):
        return self.size
    def __getitem__(self, index):
        return self.data[index]

In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch.multiprocessing as mp
import multiprocess as multi
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import os


def ddp_setup(rank, world_size):
    """
    Args:
        rank: Unique identifier of each process
        world_size: Total number of processes
    """
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12355"
    torch.cuda.set_device(rank)
    init_process_group(backend="nccl", rank=rank, world_size=world_size)


In [3]:
class Trainer:
    def __init__(
        self,
        model: torch.nn.Module,
        train_data: DataLoader,
        optimizer: torch.optim.Optimizer,
        gpu_id: int,
        save_every: int,
    ) -> None:
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_data = train_data
        self.optimizer = optimizer
        self.save_every = save_every
        self.model = DDP(model, device_ids=[gpu_id])

    def _run_batch(self, source, targets):
        self.optimizer.zero_grad()
        output = self.model(source)
        loss = F.cross_entropy(output, targets)
        loss.backward()
        self.optimizer.step()

    def _run_epoch(self, epoch):
        b_sz = len(next(iter(self.train_data))[0])
        print(f"[GPU{self.gpu_id}] Epoch {epoch} | Batchsize: {b_sz} | Steps: {len(self.train_data)}")
        self.train_data.sampler.set_epoch(epoch)
        for source, targets in self.train_data:
            source = source.to(self.gpu_id)
            targets = targets.to(self.gpu_id)
            self._run_batch(source, targets)

    def _save_checkpoint(self, epoch):
        ckp = self.model.module.state_dict()
        PATH = "checkpoint.pt"
        torch.save(ckp, PATH)
        print(f"Epoch {epoch} | Training checkpoint saved at {PATH}")

    def train(self, max_epochs: int):
        for epoch in range(max_epochs):
            self._run_epoch(epoch)
            if self.gpu_id == 0 and epoch % self.save_every == 0:
                #self._save_checkpoint(epoch)
                print('sve')

In [4]:
def load_train_objs():
    train_set = MyTrainDataset(2048)  # load your dataset
    model = torch.nn.Linear(20, 1)  # load your model
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    return train_set, model, optimizer


def prepare_dataloader(dataset: Dataset, batch_size: int):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        sampler=DistributedSampler(dataset)
    )


def pro(rank: int, world_size: int = 2, save_every: int = 10, total_epochs: int = 500, batch_size: int = 128):
    ddp_setup(rank, world_size)
    dataset, model, optimizer = load_train_objs()
    train_data = prepare_dataloader(dataset, batch_size)
    trainer = Trainer(model, train_data, optimizer, rank, save_every)
    trainer.train(total_epochs)
    destroy_process_group()

In [5]:
world_size = torch.cuda.device_count()
save_every = 10
total_epochs = 500
batch_size = 128

In [6]:
with multi.Pool(processes=world_size) as pool:
    pool.map(pro,range(world_size))

[GPU0] Epoch 0 | Batchsize: 128 | Steps: 8[GPU1] Epoch 0 | Batchsize: 128 | Steps: 8

sve[GPU1] Epoch 1 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 1 | Batchsize: 128 | Steps: 8
[GPU1] Epoch 2 | Batchsize: 128 | Steps: 8[GPU0] Epoch 2 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 3 | Batchsize: 128 | Steps: 8[GPU1] Epoch 3 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 4 | Batchsize: 128 | Steps: 8[GPU1] Epoch 4 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 5 | Batchsize: 128 | Steps: 8[GPU1] Epoch 5 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 6 | Batchsize: 128 | Steps: 8[GPU1] Epoch 6 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 7 | Batchsize: 128 | Steps: 8[GPU1] Epoch 7 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 8 | Batchsize: 128 | Steps: 8[GPU1] Epoch 8 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 9 | Batchsize: 128 | Steps: 8[GPU1] Epoch 9 | Batchsize: 128 | Steps: 8

[GPU0] Epoch 10 | Batchsize: 128 | Steps: 8[GPU1] Epoch 10 | Batchsize: 128 | Steps: 8

sve
[GPU1] Epoch 11 | Batchsize: 128 | Steps: 8[

In [7]:
#mp.spawn(pro, args=(world_size, save_every, total_epochs, batch_size), nprocs=world_size)